In [ ]:
"""
프로젝트 진행 기본 코드
"""

import pandas as pd
import datetime as dt


# 날짜 차이 계산
def calculate_days_difference(date):
    NOW = dt.datetime(2022, 12, 1)
    # 현재 날짜와 입력 날짜 간의 차이를 계산
    difference = NOW - date
    return difference.days


# 데이터 파일 불러오기
def read_data(path):
    df = pd.read_csv(path)

    # 주문일자 컬럼 datetime 자료형으로 변환
    df["주문일자"] = pd.to_datetime(df["주문일자"])
    return df


# 인자로 전달한 dataframe을  csv 파일로 저장
def save_dataframe(dataframe: pd.DataFrame):
    dataframe.to_csv("RFM_report.csv")

def save_dataframe2(dataframe: pd.DataFrame):
    dataframe.to_csv("df.csv")


file_path = "./shopping.csv"

df = read_data(file_path)

In [ ]:
# 데이터확인
print(df.head(10))
print(df.tail(10))

In [ ]:
print(f"데이터 형태 : {df.shape}")
print(f"결측치 확인 : {df.isnull().sum()}")
print(f"중복데이터 확인 : {df.duplicated().value_counts()}")
print(f"데이터 자료형 : {df.info()}")
print(f"처리상태 : {df['처리상태'].unique()}")
print(f"결제방법 : {df['결제방법'].unique()}")

In [ ]:
## 결측치 처리
# 결측치가 발생한 사람
print(f"결측치가 발생한 사람 : {df[df['상품명'].isnull()].loc[:,'구매자'].unique()}")
print(
    f"결측치가 발생했을 때 처리상태 : {df[df['상품명'].isnull()].loc[:,'처리상태'].unique()}"
)
print(
    f"결측치가 발생했을 때 처리상태 : {df[df['상품명'].isnull()].loc[:,'처리상태'].unique()}"
)

In [ ]:
df[df["판매금액"] == 0]

In [ ]:
## 데이터 전처리

# 처리상태
"""
처리상태가 주문취소, 미결제, 환불완료,환불승인인 항목은 제거할 것 -> 
확실히 구매의사를 철회했다고 판단되는 항목에 한해서 제거, 후불반려는 구매의사는 있지만 개인 금전과 관련된 부분이고 부분 환불이나 취소 역시 
구매의사가 있었다고 판단.
"""

# 처리상태 전처리
df = df[
    df["처리상태"].isin(
        [
            "구매확정",
            "후불반려",
            "부분환불",
            "부분취소",
            "상담취소",
            "교환완료",
            "상담형상품접수",
            "환불요청",
            "배송완료",
            "배송준비",
            "배송중",
            "시안확인요청",
            "결제완료",
        ]
    )
].reset_index(drop=True)

# 결측치(상품명)
"""확인결과 상품명이 결측치일때 주문금액이 다르고 결제수단, 구매확정 등 다른 요소가 여전히 존재하기에 본 분석에서는 제거할 수 없는 것으로 확인했고 상품명이 결측치인 경우 unidentify라는 이름으로 대체하여 분석 진행"""

# 상품명 전처리
df["상품명"] = df["상품명"].fillna("unidentify")

# 결제방법
"""
위 과정을 다 거치고 남은 결제 방법이 결측치인것을 확인해본 결과 구매자가 구매를 한 항목이라기 보다는 사은품이나 가맹용 제품인 경우, 즉 이 값들은 제거한다
"""

# 결제방법 전처리
df = df.dropna()


print(f"최종 결측치 확인 : {df.isnull().sum()}")


# 추가 전처리
"""판매금액이 0원인 것은 부분취소 혹은 환불이거나 학습 교구등인 경우가 있는데 실제 금액을 다루기가 어려움으로 제거하고 분석"""
df = df[df["판매금액"] != 0]

# 최종데이터
print(f"분석에 사용할 최종 데이터 형태 : {df.shape}")

In [ ]:
## RFM_result 생성

"""
- 구매자 (그룹화 기준)
- recency (최근 구매일)
- frequency (총 구매 횟수)
- monetary (총 구매 금액)
- R_score (Recency 점수 1 ~ 5)
- F_score (Frequency 점수 1 ~ 5)
- M_score (Monetary 점수 1 ~ 5)
- grade (RFM 점수를 기준으로 분류한 5개 그룹 중 하나)
"""

# 구매자 그룹
grouped = df.groupby("구매자")

recency = grouped["주문일자"].max()
frequency = grouped["주문번호"].count()
monetary = grouped["판매금액"].sum()

# R_score
daylen = []
for i in range(len(recency)):
    a = calculate_days_difference(recency[i])
    daylen.append(a)

R_score = []
for v in daylen:
    if v <= 30:
        v = 5
    elif v <= 90:
        v = 4
    elif v <= 180:
        v = 3
    elif v <= 365:
        v = 2
    else:
        v = 1
    R_score.append(v)

print(R_score)

# F_score
F_score = pd.qcut(
    frequency,
    q=[0, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=["1", "2", "3", "4", "5"],
)

print(F_score)


# M_score
M_score = pd.qcut(
    monetary,
    q=[0, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=["1", "2", "3", "4", "5"],
)

print(M_score)

In [ ]:
## 신규 데이터 프레임 형성
RFM_result = pd.DataFrame(data=recency)
RFM_result["frequency"] = frequency.values
RFM_result["monetary"] = monetary.values
RFM_result["R_score"] = R_score
RFM_result["F_score"] = F_score.values
RFM_result["M_score"] = M_score.values
RFM_result["Grade"] = ""

RFM_result

## **Grade** 나누기
### Vip 
    5,5,5
    5,4,5
    4,5,5
    4,4,5
    R이 4 이상이면서 F,M 합이 9이상(단 M이 5일것)
### 이탈우려고객
    R이 2,3이면서 F,M의 합이 5이상
### 이탈고객
    R이 1이면서 F,M의 합이 5이상
### 신경써야할고객(이탈위험, 구매빈도에 비해 지출액이 적은 고객)
    R이 4이상이면서 F,M의 합이 6이상
### 신규고객
    R과 F가 1인 고객

In [ ]:
for i in range(len(RFM_result)):
    if RFM_result.iloc[i, 3] >= 4:
        if int(RFM_result.iloc[i, 4]) >= 4:
            if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 9:
                RFM_result.iloc[i, 6] = "Vip"
            elif int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 6:
                RFM_result.iloc[i, 6] = "신경써야할고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"
    elif RFM_result.iloc[i, 3] >= 2:
        if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 5:
            RFM_result.iloc[i, 6] = "이탈우려고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"
    elif RFM_result.iloc[i, 3] == 1:
        if int(RFM_result.iloc[i, 4]) + int(RFM_result.iloc[i, 5]) >= 5:
            RFM_result.iloc[i, 6] = "이탈고객"
        elif int(RFM_result.iloc[i, 5]) == 1:
            RFM_result.iloc[i, 6] = "신규고객"
        else:
            RFM_result.iloc[i, 6] = "신경써야할고객"


# 이탈우려고객


# int(RFM_result.iloc[i,4])+int(RFM_result.iloc[i,5])

In [ ]:
save_dataframe(RFM_result)

In [ ]:
save_dataframe2(df)